# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with 

mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
mongo.list_database_names()

['admin',
 'class_db',
 'config',
 'epa',
 'establishments',
 'fruits_db',
 'local',
 'met',
 'petsitly_marketing',
 'test',
 'uk_food']

In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
db.list_collection_names()

['establishments']

In [6]:
# review a document in the establishments collection
document = db['establishments'].find_one()
document

{'_id': ObjectId('64691da9aaf3ed47b60b5c99'),
 'FHRSID': 1349980,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000084064',
 'BusinessName': 'Boodles',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': '3 The Droveway',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT15 6DH',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2021-06-14T00:00:00',
 'LocalAuthorityCode': '182',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'scores': {'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.378536', 'latitude': '51.150879'},
 'RightToReply': '',
 'Distance': 4587.638929767669,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount': 0,
  'returncode':

In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_restaurant = {"BusinessName":"Penang Flavours"}

In [9]:
# ADDED: This is to remove any extras from subsequent runs of the code
establishments.delete_many(new_restaurant)

In [10]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

In [11]:
# Check that the new restaurant was inserted
results = establishments.find({"BusinessName":"Penang Flavours"})
for result in results:
    pprint(result)

{'BusinessName': 'Penang Flavours', '_id': ObjectId('64691dd19dcc7ea0f16e2f3d')}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
filter = {"BusinessType":"Restaurant/Cafe/Canteen"}
projection = {"BusinessType":1,"BusinessTypeID":1}
result = establishments.find_one(filter,projection)
result

{'_id': ObjectId('64691da9aaf3ed47b60b5c99'),
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1}

3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID
filter = {"BusinessName":"Penang Flavours"}
update = {"$set":{
    "BusinessTypeID":"1",
    "BusinessType":"Restaurant/Cafe/Canteen"}
         }
result = establishments.update_one(filter,update)

In [14]:
# Confirm that the new restaurant was updated
results = establishments.find({"BusinessName":"Penang Flavours"})
for result in results:
    pprint(result)

{'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '1',
 '_id': ObjectId('64691dd19dcc7ea0f16e2f3d')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
filter = {"LocalAuthorityName":"Dover"}
count = establishments.count_documents(filter)
count

994

In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
result = establishments.delete_many(filter)

In [17]:
# Check if any remaining documents include Dover
filter = {"LocalAuthorityName":"Dover"}
count = establishments.count_documents(filter)
count

0

In [18]:
# Check that other documents remain with 'find_one'
document = db['establishments'].find_one()
document

{'_id': ObjectId('64691da9aaf3ed47b60b5f7c'),
 'FHRSID': 647177,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT19 6PY',
 'Phone': '',
 'RatingValue': '4',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingDate': '2014-03-31T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.196408', 'latitude': '51.086058'},
 'RightToReply': '',
 'Distance': 4591.821311183521,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  '

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [19]:
# ADDED: Ran this to see what the datatype was for the lat and long fields before making the change
document = db['establishments'].find_one()
longitude = document['geocode']['longitude']
latitude = document['geocode']['latitude']
longitude_type = type(longitude).__name__
latitude_type = type(latitude).__name__
print(longitude_type)
print(latitude_type)

str
str


In [20]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many(
    {
        'geocode.latitude': {'$exists': True, '$type': 'string'},
        'geocode.longitude': {'$exists': True, '$type': 'string'}
    }, 
    [
        {
            '$set': {
                'geocode.latitude': { '$toDouble': '$geocode.latitude'},
                'geocode.longitude': { '$toDouble': '$geocode.longitude'}
            }
        }
    ]
)

Use `update_many` to convert `RatingValue` to integer numbers.

In [21]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [22]:
# ADDED:This let's me see what the RatingValue type was first
document = db['establishments'].find_one()
rating_value = document['RatingValue']
rating_value_type = type(rating_value).__name__
print(rating_value_type)

str


In [23]:
# Change the data type from String to Integer for RatingValue
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {
        'RatingValue': {'$exists': True, '$type': 'string'}
    }, 
    [
        {
            '$set': {
                'RatingValue': { '$toInt': '$RatingValue'}
            }
        }
    ]
)

In [24]:
# Check that the coordinates and rating value are now numbers
document = db['establishments'].find_one()
longitude = document['geocode']['longitude']
latitude = document['geocode']['latitude']
longitude_type = type(longitude).__name__
latitude_type = type(latitude).__name__
print(longitude_type)
print(latitude_type)

rating_value = document['RatingValue']
rating_value_type = type(rating_value).__name__
print(rating_value_type)

float
float
int
